In [1]:
from datetime import datetime

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from scipy import stats

plt.style.use('ggplot')

In [3]:
def load_and_process_data(excel_file):
    """
    Carga los datos del archivo Excel y procesa cada hoja como un cliente diferente
    """
    client_dfs = {}
    
    excel = pd.ExcelFile(excel_file)
    
    for cliente in excel.sheet_names:
        df = pd.read_excel(excel, sheet_name=cliente)
        
        df['Fecha'] = pd.to_datetime(df['Fecha'])
        
        df['hora'] = df['Fecha'].dt.hour
        df['dia_semana'] = df['Fecha'].dt.dayofweek
        df['mes'] = df['Fecha'].dt.month
        
        client_dfs[cliente] = df
    
    return client_dfs

client_dfs = load_and_process_data('Datos_Contugas.xlsx')

In [5]:
import pandas as pd
from datetime import datetime
from IPython.display import display

# Definir rangos de valores permitidos para detectar anomalías
min_presion, max_presion = 10, 100  # Ejemplo de rango de presión
min_temperatura, max_temperatura = -10, 50  # Ejemplo de rango de temperatura
min_volumen, max_volumen = 0, 200  # Ejemplo de rango de volumen

rango_presion = (min_presion, max_presion)
rango_temperatura = (min_temperatura, max_temperatura)
rango_volumen = (min_volumen, max_volumen)

# Lista para almacenar los resultados del resumen
resumen_datos = []

# Función para generar el resumen por cliente y mostrarlo en el notebook
def generar_resumen_cliente(df, nombre_hoja):
    # Convertir la columna 'Fecha' a tipo datetime
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    
    # 1. Granularidad en horas
    granularidad = df['Fecha'].diff().min().total_seconds() / 3600

    # 2. Valores anómalos
    anomalos_presion = df[(df['Presion'] < rango_presion[0]) | (df['Presion'] > rango_presion[1])]
    anomalos_temperatura = df[(df['Temperatura'] < rango_temperatura[0]) | (df['Temperatura'] > rango_temperatura[1])]
    anomalos_volumen = df[(df['Volumen'] < rango_volumen[0]) | (df['Volumen'] > rango_volumen[1])]
    
    num_anomalos_presion = len(anomalos_presion)
    num_anomalos_temperatura = len(anomalos_temperatura)
    num_anomalos_volumen = len(anomalos_volumen)
    
    # 3. Registros duplicados
    registros_duplicados = df.duplicated().sum()

    # 4. Porcentaje de datos faltantes
    total_faltantes = df.isnull().sum().sum()
    porcentaje_faltantes = (total_faltantes / df.size) * 100

    # 5. Edad promedio de los datos en días
    df['Edad_Datos'] = (datetime.now() - df['Fecha']).dt.days
    edad_promedio = df['Edad_Datos'].mean()
    
    # 6. Correlación promedio entre variables clave
    correlacion = df[['Presion', 'Temperatura', 'Volumen']].corr().mean().mean()
    
    # Crear un DataFrame para mostrar el resumen de cada cliente
    resumen_cliente = pd.DataFrame([{
        'Cliente': nombre_hoja,
        'Granularidad (hrs)': granularidad,
        'Anomalías Presión': num_anomalos_presion,
        'Anomalías Temperatura': num_anomalos_temperatura,
        'Anomalías Volumen': num_anomalos_volumen,
        'Duplicados': registros_duplicados,
        'Faltantes (%)': round(porcentaje_faltantes, 2),
        'Edad Promedio (días)': round(edad_promedio, 2),
        'Correlación Promedio': round(correlacion, 2)
    }])
    
    # Mostrar el resumen del cliente en el notebook
    display(resumen_cliente)

# Aplicar la función a todas las hojas que comienzan con "CLIENTE"
for hoja, df in client_dfs.items():
    if hoja.startswith('CLIENTE'):
        generar_resumen_cliente(df, hoja)

,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE1,1.0,0,0,0,0,0.0,1228.07,0.38


,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE2,0.0,0,0,3,0,0.0,1213.48,0.34


,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE3,0.0,42248,0,140,0,0.0,1231.0,0.19


,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE4,1.0,0,0,2400,0,0.0,1222.55,0.31


,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE5,1.0,0,0,0,0,0.0,1227.76,0.31


...

,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE16,-1981.0,0,0,25099,0,0.0,1225.91,0.17


,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE17,1.0,0,0,0,0,0.0,1228.07,0.36


,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE18,0.0,0,0,6,0,0.0,1213.48,0.35


,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE19,1.0,0,0,2254,0,0.0,1222.55,0.31


,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE20,1.0,0,0,22105,0,0.0,1226.44,0.4


In [8]:
# Verificar duplicados y valores nulos en los datos de cada cliente
resultados_validacion = []

for cliente, df in client_dfs.items():
    nulos = df.isnull().sum().sum()
    duplicados = df.duplicated().sum()
    resultados_validacion.append({
        "Cliente": cliente,
        "Nulos": nulos,
        "Duplicados": duplicados
    })

# Convertir a DataFrame para tabular los resultados
resumen_validacion = pd.DataFrame(resultados_validacion)

# Mostrar el resumen en el notebook
print("Validación de Datos")
print(resumen_validacion)

# Verificar si hay duplicados o nulos en los datos
if resumen_validacion["Nulos"].sum() == 0 and resumen_validacion["Duplicados"].sum() == 0:
    print("\nNo se encontraron duplicados ni valores nulos en los datos de los 20 clientes.")
else:
    print("\nSe detectaron duplicados o valores nulos en los datos.")


Validación de Datos
      Cliente  Nulos  Duplicados
0    CLIENTE1      0           0
1    CLIENTE2      0           0
2    CLIENTE3      0           0
3    CLIENTE4      0           0
4    CLIENTE5      0           0
5    CLIENTE6      0           0
6    CLIENTE7      0           0
7    CLIENTE8      0           0
8    CLIENTE9      0           0
9   CLIENTE10      0           0
10  CLIENTE11      0           0
11  CLIENTE12      0           0
12  CLIENTE13      0           0
13  CLIENTE14      0           0
14  CLIENTE15      0           0
15  CLIENTE16      0           0
16  CLIENTE17      0           0
17  CLIENTE18      0           0
18  CLIENTE19      0           0
19  CLIENTE20      0           0

No se encontraron duplicados ni valores nulos en los datos de los 20 clientes.


In [10]:
import pandas as pd
from datetime import datetime
from IPython.display import display

# Definir rangos de valores permitidos para detectar anomalías
min_presion, max_presion = 10, 100  # Ejemplo de rango de presión
min_temperatura, max_temperatura = -10, 50  # Ejemplo de rango de temperatura
min_volumen, max_volumen = 0, 200  # Ejemplo de rango de volumen

rango_presion = (min_presion, max_presion)
rango_temperatura = (min_temperatura, max_temperatura)
rango_volumen = (min_volumen, max_volumen)

# Lista para almacenar los resultados del resumen
resumen_datos = []

# Función para generar el resumen por cliente y mostrarlo en el notebook
def generar_resumen_cliente(df, nombre_hoja):
    # Convertir la columna 'Fecha' a tipo datetime
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    
    # 1. Calcular Granularidad en horas
    granularidad = df['Fecha'].diff().min().total_seconds() / 3600
    df['Granularidad (hrs)'] = granularidad  # Agregar la columna 'Granularidad (hrs)'
    
    # 2. Valores anómalos
    anomalos_presion = df[(df['Presion'] < rango_presion[0]) | (df['Presion'] > rango_presion[1])]
    anomalos_temperatura = df[(df['Temperatura'] < rango_temperatura[0]) | (df['Temperatura'] > rango_temperatura[1])]
    anomalos_volumen = df[(df['Volumen'] < rango_volumen[0]) | (df['Volumen'] > rango_volumen[1])]
    
    num_anomalos_presion = len(anomalos_presion)
    num_anomalos_temperatura = len(anomalos_temperatura)
    num_anomalos_volumen = len(anomalos_volumen)
    
    # 3. Registros duplicados
    registros_duplicados = df.duplicated().sum()

    # 4. Porcentaje de datos faltantes
    total_faltantes = df.isnull().sum().sum()
    porcentaje_faltantes = (total_faltantes / df.size) * 100

    # 5. Edad promedio de los datos en días
    df['Edad_Datos'] = (datetime.now() - df['Fecha']).dt.days
    edad_promedio = df['Edad_Datos'].mean()
    
    # 6. Correlación promedio entre variables clave
    correlacion = df[['Presion', 'Temperatura', 'Volumen']].corr().mean().mean()
    
    # Crear un DataFrame para mostrar el resumen de cada cliente
    resumen_cliente = pd.DataFrame([{
        'Cliente': nombre_hoja,
        'Granularidad (hrs)': granularidad,
        'Anomalías Presión': num_anomalos_presion,
        'Anomalías Temperatura': num_anomalos_temperatura,
        'Anomalías Volumen': num_anomalos_volumen,
        'Duplicados': registros_duplicados,
        'Faltantes (%)': round(porcentaje_faltantes, 2),
        'Edad Promedio (días)': round(edad_promedio, 2),
        'Correlación Promedio': round(correlacion, 2)
    }])
    
    # Mostrar el resumen del cliente en el notebook
    display(resumen_cliente)

# Aplicar la función a todas las hojas que comienzan con "CLIENTE"
for hoja, df in client_dfs.items():
    if hoja.startswith('CLIENTE'):
        generar_resumen_cliente(df, hoja)

# Si quieres identificar clientes con granularidad negativa
clientes_con_granularidad_negativa = {}
for cliente, df in client_dfs.items():
    if 'Granularidad (hrs)' in df.columns:
        df_negativo = df[df['Granularidad (hrs)'] < 0]
        if not df_negativo.empty:
            clientes_con_granularidad_negativa[cliente] = df_negativo

# Mostrar los clientes con granularidad negativa
for cliente, df_negativo in clientes_con_granularidad_negativa.items():
    print(f"Cliente: {cliente}")
    print(df_negativo[['Fecha', 'Granularidad (hrs)']])  # Mostrar solo la fecha y la granularidad


,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE1,1.0,0,0,0,0,0.0,1228.11,0.38


,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE2,0.0,0,0,3,0,0.0,1213.52,0.34


,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE3,0.0,42248,0,140,0,0.0,1231.04,0.19


,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE4,1.0,0,0,2400,0,0.0,1222.59,0.31


,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE5,1.0,0,0,0,0,0.0,1227.8,0.31


...

,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE17,1.0,0,0,0,0,0.0,1228.11,0.36


,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE18,0.0,0,0,6,0,0.0,1213.52,0.35


,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE19,1.0,0,0,2254,0,0.0,1222.59,0.31


,Cliente,Granularidad (hrs),Anomalías Presión,Anomalías Temperatura,Anomalías Volumen,Duplicados,Faltantes (%),Edad Promedio (días),Correlación Promedio
0,CLIENTE20,1.0,0,0,22105,0,0.0,1226.48,0.4


Cliente: CLIENTE8
                    Fecha  Granularidad (hrs)
0     2019-01-14 00:00:00             -1981.0
1     2019-01-14 01:00:00             -1981.0
2     2019-01-14 02:00:00             -1981.0
3     2019-01-14 03:00:00             -1981.0
4     2019-01-14 04:00:00             -1981.0
...                   ...                 ...
43142 2023-12-31 19:00:00             -1981.0
43143 2023-12-31 20:00:00             -1981.0
43144 2023-12-31 21:00:00             -1981.0
43145 2023-12-31 22:00:00             -1981.0
43146 2023-12-31 23:00:00             -1981.0

[43147 rows x 2 columns]
Cliente: CLIENTE16
                    Fecha  Granularidad (hrs)
0     2019-01-14 00:00:00             -1981.0
1     2019-01-14 01:00:00             -1981.0
2     2019-01-14 02:00:00             -1981.0
3     2019-01-14 03:00:00             -1981.0
4     2019-01-14 04:00:00             -1981.0
...                   ...                 ...
43142 2023-12-31 19:00:00             -1981.0
43143 2023-12-31 

In [12]:
import pandas as pd

#Verificar el formato de las fechas en el DataFrame
def verificar_formatos_fechas(df):
    # Intentar convertir todas las fechas a datetime y registrar las que fallen
    fechas_invalidas = []
    for idx, fecha in enumerate(df['Fecha']):
        try:
#Convertir la fecha a formato datetime
            pd.to_datetime(fecha)
        except Exception as e:
            # Si ocurre un error, registramos la fila con fecha inválida
            fechas_invalidas.append((idx, fecha, str(e)))
    
    #fechas inválidas
    if fechas_invalidas:
        print(f"Se encontraron {len(fechas_invalidas)} fechas con formato incorrecto:")
        for idx, fecha, error in fechas_invalidas:
            print(f"Índice: {idx}, Fecha: {fecha}, Error: {error}")
    else:
        print("Todas las fechas tienen un formato correcto.")


for hoja, df in client_dfs.items():
    if hoja.startswith('CLIENTE'):
        print(f"\nVerificando fechas en {hoja}...")
        verificar_formatos_fechas(df)



Verificando fechas en CLIENTE1...
Todas las fechas tienen un formato correcto.

Verificando fechas en CLIENTE2...
Todas las fechas tienen un formato correcto.

Verificando fechas en CLIENTE3...
Todas las fechas tienen un formato correcto.

Verificando fechas en CLIENTE4...
Todas las fechas tienen un formato correcto.

Verificando fechas en CLIENTE5...
Todas las fechas tienen un formato correcto.

Verificando fechas en CLIENTE6...
Todas las fechas tienen un formato correcto.

Verificando fechas en CLIENTE7...
Todas las fechas tienen un formato correcto.

Verificando fechas en CLIENTE8...
Todas las fechas tienen un formato correcto.

Verificando fechas en CLIENTE9...
Todas las fechas tienen un formato correcto.

Verificando fechas en CLIENTE10...
Todas las fechas tienen un formato correcto.

Verificando fechas en CLIENTE11...
Todas las fechas tienen un formato correcto.

Verificando fechas en CLIENTE12...
Todas las fechas tienen un formato correcto.

Verificando fechas en CLIENTE13...
T

In [13]:
# Granularidad en horas, ordenando las fechas
def calcular_granularidad(df):
    # Ordenar las fechas
    df = df.sort_values(by='Fecha')
    
 #Diferencia en horas entre fechas consecutivas
    granularidad = df['Fecha'].diff().min().total_seconds() / 3600
    return granularidad
for hoja, df in client_dfs.items():
    if hoja.startswith('CLIENTE'):
        granularidad = calcular_granularidad(df)
        print(f"Granularidad para {hoja}: {granularidad} horas")


Granularidad para CLIENTE1: 1.0 horas
Granularidad para CLIENTE2: 0.0 horas
Granularidad para CLIENTE3: 0.0 horas
Granularidad para CLIENTE4: 1.0 horas
Granularidad para CLIENTE5: 1.0 horas
Granularidad para CLIENTE6: 1.0 horas
Granularidad para CLIENTE7: 1.0 horas
Granularidad para CLIENTE8: 0.0 horas
Granularidad para CLIENTE9: 1.0 horas
Granularidad para CLIENTE10: 1.0 horas
Granularidad para CLIENTE11: 0.0 horas
Granularidad para CLIENTE12: 1.0 horas
Granularidad para CLIENTE13: 1.0 horas
Granularidad para CLIENTE14: 1.0 horas
Granularidad para CLIENTE15: 1.0 horas
Granularidad para CLIENTE16: 0.0 horas
Granularidad para CLIENTE17: 1.0 horas
Granularidad para CLIENTE18: 0.0 horas
Granularidad para CLIENTE19: 1.0 horas
Granularidad para CLIENTE20: 1.0 horas
